# Treasury Copilot - GenAI Assistant

## Use Case Overview

**Business Problem:** Treasury teams spend significant time on:
- Answering repetitive questions about policies
- Generating routine reports
- Explaining data and variances
- Looking up historical information

**AI Solution:** A GenAI-powered assistant that:
1. Answers questions using treasury policies (RAG)
2. Queries treasury data in natural language
3. Generates narrative explanations
4. Provides recommendations based on context

---

## Architecture

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│   User      │────▶│  Copilot    │────▶│    LLM      │
│   Query     │     │  (RAG)      │     │  (GPT-4)    │
└─────────────┘     └──────┬──────┘     └─────────────┘
                           │
              ┌────────────┼────────────┐
              ▼            ▼            ▼
        ┌──────────┐ ┌──────────┐ ┌──────────┐
        │ Policy   │ │ Treasury │ │ Market   │
        │ Docs     │ │ Data     │ │ Data     │
        └──────────┘ └──────────┘ └──────────┘
```

---

## Learning Objectives

1. Build a simple RAG system for policy Q&A
2. Create prompt templates for treasury queries
3. Demonstrate LLM integration patterns
4. Build a functional chatbot interface

## 1. Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
from datetime import datetime
import json
import os

import sys
sys.path.append('..')
from src.treasury_sim.generators import (
    generate_cash_flows, generate_fx_rates, 
    generate_daily_cash_position, set_seed
)

print("✅ Setup complete!")

## 2. Create Sample Knowledge Base

### Thinking Trace 🧠

> **RAG (Retrieval Augmented Generation) works by:**
> 1. Storing documents in a vector database
> 2. Converting user query to embedding
> 3. Finding relevant document chunks
> 4. Passing context + query to LLM
>
> For this demo, we'll simulate this with a simple keyword-based retrieval.

In [ ]:
# Sample treasury policy documents
POLICY_DOCUMENTS = {
    "cash_management": """
    CASH MANAGEMENT POLICY
    
    1. MINIMUM CASH BALANCE
    - Operating accounts: Minimum $5 million USD equivalent
    - Payroll accounts: 1.5x monthly payroll requirement
    - Tax accounts: Maintain 110% of quarterly tax liability
    
    2. EXCESS CASH INVESTMENT
    - Amounts over $10M can be invested in money market funds
    - Maximum maturity: 90 days
    - Approved counterparties: A-rated banks only
    
    3. BORROWING GUIDELINES
    - Utilize revolving credit facility first
    - Commercial paper for amounts > $50M
    - Notify CFO for borrowings exceeding $100M
    """,
    
    "fx_policy": """
    FOREIGN EXCHANGE POLICY
    
    1. HEDGE RATIOS
    - Committed exposures: 100% hedged
    - Forecasted exposures (0-6 months): 75% hedged
    - Forecasted exposures (6-12 months): 50% hedged
    
    2. APPROVED INSTRUMENTS
    - FX Forwards: Approved for all maturities up to 12 months
    - FX Options: Requires Treasury Director approval
    - Cross-currency swaps: CFO approval required
    
    3. COUNTERPARTY LIMITS
    - Maximum $50M per counterparty for FX trades
    - Only A-rated or higher banks
    """,
    
    "payment_policy": """
    PAYMENT AUTHORIZATION POLICY
    
    1. APPROVAL LIMITS
    - Up to $50,000: Manager approval
    - $50,001 - $500,000: Director approval
    - $500,001 - $5,000,000: VP Treasury approval
    - Above $5,000,000: CFO approval required
    
    2. DUAL AUTHORIZATION
    - All payments above $100,000 require dual authorization
    - Dual authorization from same department not allowed
    
    3. PAYMENT TIMING
    - Standard payments: Net 30 days
    - Early payment discounts: Evaluate if >2% discount offered
    - Late payment: Requires written justification
    """,
    
    "counterparty_policy": """
    COUNTERPARTY RISK POLICY
    
    1. CREDIT LIMITS
    - AAA rated: $100M limit
    - AA rated: $75M limit
    - A rated: $50M limit
    - BBB rated: $25M limit (requires approval)
    - Below BBB: Not permitted
    
    2. MONITORING
    - Review credit ratings quarterly
    - Alert on any rating downgrade
    - Review exposure concentrations monthly
    """
}

print(f"📚 Loaded {len(POLICY_DOCUMENTS)} policy documents")

In [ ]:
# Simple retrieval function (simulates vector search)
def retrieve_relevant_policies(query, top_k=2):
    """
    Simple keyword-based retrieval.
    In production, this would use embeddings + vector DB.
    """
    query_lower = query.lower()
    
    # Keyword mapping
    keyword_map = {
        'cash_management': ['cash', 'balance', 'minimum', 'investment', 'borrowing', 'liquidity'],
        'fx_policy': ['fx', 'foreign', 'exchange', 'hedge', 'currency', 'forward', 'exposure'],
        'payment_policy': ['payment', 'approval', 'authorize', 'limit', 'dual'],
        'counterparty_policy': ['counterparty', 'credit', 'rating', 'limit', 'bank']
    }
    
    # Score each document
    scores = {}
    for doc_name, keywords in keyword_map.items():
        score = sum(1 for kw in keywords if kw in query_lower)
        scores[doc_name] = score
    
    # Return top-k
    sorted_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    relevant = [doc for doc, score in sorted_docs[:top_k] if score > 0]
    
    if not relevant:
        relevant = ['cash_management']  # Default
    
    return [POLICY_DOCUMENTS[doc] for doc in relevant]

# Test retrieval
test_query = "What is the minimum cash balance requirement?"
results = retrieve_relevant_policies(test_query)
print(f"Query: {test_query}")
print(f"Retrieved {len(results)} relevant document(s)")

## 3. Load Treasury Data

In [ ]:
# Generate sample treasury data
set_seed(42)

cash_flows = generate_cash_flows(days=90, n_accounts=3, seed=42)
daily_positions = generate_daily_cash_position(days=90, n_accounts=3, seed=42)
fx_rates = generate_fx_rates(days=90, currency_pairs=['USD/TRY', 'EUR/TRY'], seed=42)

# Aggregate for queries
daily_cash = daily_positions.groupby('date').agg({
    'opening_balance': 'sum',
    'closing_balance': 'sum',
    'inflows': 'sum',
    'outflows': 'sum'
}).reset_index()

print("📊 Treasury Data Loaded")
print(f"   - Cash flows: {len(cash_flows):,} transactions")
print(f"   - Daily positions: {len(daily_positions):,} records")
print(f"   - FX rates: {len(fx_rates):,} rates")

## 4. Create Treasury Query Functions

### Thinking Trace 🧠

> **Function calling pattern:**
> 1. LLM understands user intent
> 2. LLM calls appropriate function with parameters
> 3. Function returns data
> 4. LLM formats response in natural language

In [ ]:
# Treasury query functions
def get_current_cash_position():
    """Get the latest cash position."""
    latest = daily_cash.iloc[-1]
    return {
        'date': str(latest['date'].date()),
        'total_balance': f"${latest['closing_balance']:,.0f}",
        'daily_inflows': f"${latest['inflows']:,.0f}",
        'daily_outflows': f"${latest['outflows']:,.0f}"
    }

def get_cash_forecast(days=7):
    """Get simple cash forecast based on historical average."""
    avg_daily_net = (daily_cash['inflows'] - daily_cash['outflows']).mean()
    current_balance = daily_cash['closing_balance'].iloc[-1]
    
    forecast = []
    balance = current_balance
    for i in range(1, days+1):
        balance += avg_daily_net
        forecast.append({
            'day': i,
            'projected_balance': f"${balance:,.0f}"
        })
    return forecast

def get_fx_rate(currency_pair='USD/TRY'):
    """Get the latest FX rate."""
    latest = fx_rates[fx_rates['currency_pair'] == currency_pair].iloc[-1]
    return {
        'pair': currency_pair,
        'rate': f"{latest['mid']:.4f}",
        'change': f"{latest['daily_change_pct']:.2f}%"
    }

def get_fx_exposure_summary():
    """Get FX exposure summary."""
    # Mock data for demo
    return {
        'USD': {'net_exposure': '-$45M', 'hedged': '72%'},
        'EUR': {'net_exposure': '+$12M', 'hedged': '85%'},
        'GBP': {'net_exposure': '-$3M', 'hedged': '100%'}
    }

# Function registry
TREASURY_FUNCTIONS = {
    'get_current_cash_position': get_current_cash_position,
    'get_cash_forecast': get_cash_forecast,
    'get_fx_rate': get_fx_rate,
    'get_fx_exposure_summary': get_fx_exposure_summary
}

print("🔧 Registered Treasury Functions:")
for func_name in TREASURY_FUNCTIONS:
    print(f"   - {func_name}")

## 5. Build the Copilot

### Thinking Trace 🧠

> **Copilot architecture:**
> 1. **Intent Classification**: Determine query type (policy, data, analysis)
> 2. **Context Retrieval**: Get relevant documents/data
> 3. **Response Generation**: Use LLM or template
>
> For this demo, we'll simulate LLM responses with templates.

In [ ]:
class TreasuryCopilot:
    """
    Treasury AI Assistant.
    
    Handles:
    - Policy questions (RAG)
    - Data queries (function calling)
    - Analysis requests
    """
    
    def __init__(self):
        self.functions = TREASURY_FUNCTIONS
        self.policies = POLICY_DOCUMENTS
        self.conversation_history = []
    
    def classify_intent(self, query):
        """Classify the query intent."""
        query_lower = query.lower()
        
        # Data queries
        if any(kw in query_lower for kw in ['current', 'balance', 'position', 'what is our']):
            if 'cash' in query_lower:
                return 'data_cash'
            elif 'fx' in query_lower or 'rate' in query_lower:
                return 'data_fx'
            elif 'exposure' in query_lower:
                return 'data_exposure'
        
        # Forecast queries
        if any(kw in query_lower for kw in ['forecast', 'predict', 'next week', 'projection']):
            return 'forecast'
        
        # Policy queries
        if any(kw in query_lower for kw in ['policy', 'limit', 'approval', 'rule', 'minimum', 'maximum', 'allowed']):
            return 'policy'
        
        return 'general'
    
    def answer_policy_question(self, query):
        """Answer policy questions using RAG."""
        # Retrieve relevant policies
        relevant_docs = retrieve_relevant_policies(query)
        context = "\n\n".join(relevant_docs)
        
        # In production, this would call an LLM
        # For demo, we'll return the relevant context with a template
        response = f"""Based on our treasury policies:

{context}

---
💡 This answer is based on the relevant policy sections above. 
Please consult with Treasury for specific situations."""
        
        return response
    
    def answer_data_query(self, query, intent):
        """Answer data queries using function calling."""
        if intent == 'data_cash':
            data = get_current_cash_position()
            return f"""📊 **Current Cash Position** (as of {data['date']})

| Metric | Value |
|--------|-------|
| Total Balance | {data['total_balance']} |
| Today's Inflows | {data['daily_inflows']} |
| Today's Outflows | {data['daily_outflows']} |

💡 Cash position is within policy limits."""
        
        elif intent == 'data_fx':
            usd = get_fx_rate('USD/TRY')
            eur = get_fx_rate('EUR/TRY')
            return f"""📊 **FX Rates** (Latest)

| Pair | Rate | Daily Change |
|------|------|-------------|
| USD/TRY | {usd['rate']} | {usd['change']} |
| EUR/TRY | {eur['rate']} | {eur['change']} |
"""
        
        elif intent == 'data_exposure':
            exposure = get_fx_exposure_summary()
            rows = "\n".join([f"| {ccy} | {data['net_exposure']} | {data['hedged']} |" 
                              for ccy, data in exposure.items()])
            return f"""📊 **FX Exposure Summary**

| Currency | Net Exposure | Hedged |
|----------|--------------|--------|
{rows}

💡 USD exposure is below target hedge ratio (75%). Consider increasing hedge."""
        
        return "I couldn't find the specific data you're looking for."
    
    def answer_forecast_query(self, query):
        """Answer forecast queries."""
        forecast = get_cash_forecast(7)
        rows = "\n".join([f"| Day {f['day']} | {f['projected_balance']} |" for f in forecast])
        
        return f"""📈 **7-Day Cash Forecast**

| Day | Projected Balance |
|-----|-------------------|
{rows}

💡 Based on 90-day historical average daily net flow.
⚠️ Note: Actual results may vary. Large known payments not included."""
    
    def chat(self, query):
        """Main chat interface."""
        # Store in history
        self.conversation_history.append({'role': 'user', 'content': query})
        
        # Classify intent
        intent = self.classify_intent(query)
        
        # Route to appropriate handler
        if intent == 'policy':
            response = self.answer_policy_question(query)
        elif intent.startswith('data_'):
            response = self.answer_data_query(query, intent)
        elif intent == 'forecast':
            response = self.answer_forecast_query(query)
        else:
            response = """I'm your Treasury AI Assistant. I can help with:

📋 **Policy Questions**
- "What is the payment approval limit?"
- "What's our FX hedge policy?"

📊 **Data Queries**
- "What's our current cash position?"
- "Show me the latest FX rates"
- "What's our FX exposure?"

📈 **Forecasts**
- "Give me a cash forecast for next week"

How can I help you today?"""
        
        # Store response
        self.conversation_history.append({'role': 'assistant', 'content': response})
        
        return response

# Initialize copilot
copilot = TreasuryCopilot()
print("🤖 Treasury Copilot initialized!")

## 6. Interactive Demo

In [ ]:
# Demo conversations
demo_queries = [
    "Hello, what can you help me with?",
    "What's our current cash position?",
    "What is the payment approval limit for $2 million?",
    "Show me the FX exposure summary",
    "What's our FX hedge policy for forecasted exposures?",
    "Give me a cash forecast for next week"
]

print("🤖 TREASURY COPILOT DEMO")
print("=" * 70)

for query in demo_queries:
    print(f"\n👤 User: {query}")
    print("\n🤖 Copilot:")
    response = copilot.chat(query)
    print(response)
    print("\n" + "-" * 70)

## 7. Production Considerations

### Architecture for Production

```
┌─────────────────────────────────────────────────────────────────────┐
│                    PRODUCTION TREASURY COPILOT                       │
├─────────────────────────────────────────────────────────────────────┤
│                                                                      │
│  ┌──────────────┐                                                   │
│  │ User Input   │                                                   │
│  └──────┬───────┘                                                   │
│         │                                                           │
│         ▼                                                           │
│  ┌──────────────┐     ┌──────────────┐     ┌──────────────┐        │
│  │   Intent     │────▶│    Router    │────▶│   Response   │        │
│  │  Detection   │     │              │     │  Generator   │        │
│  └──────────────┘     └──────────────┘     └──────────────┘        │
│                              │                     │                │
│              ┌───────────────┼───────────────┐     │                │
│              │               │               │     │                │
│              ▼               ▼               ▼     │                │
│  ┌──────────────┐ ┌──────────────┐ ┌──────────────┐│                │
│  │ Vector DB    │ │ Treasury API │ │   LLM API    ││                │
│  │ (Policies)   │ │ (SAP/TMS)    │ │ (GPT-4/Claude││                │
│  └──────────────┘ └──────────────┘ └──────────────┘│                │
│                                                     │                │
│  ┌──────────────────────────────────────────────────┘               │
│  │                                                                   │
│  ▼                                                                   │
│  ┌──────────────────────────────────────────────────────────────┐   │
│  │                   Security & Compliance                       │   │
│  │   • Role-based access    • Audit logging    • Data masking   │   │
│  └──────────────────────────────────────────────────────────────┘   │
│                                                                      │
└─────────────────────────────────────────────────────────────────────┘
```

### Key Components

| Component | Technology Options |
|-----------|-------------------|
| **Vector DB** | Azure AI Search, Pinecone, Weaviate |
| **LLM** | Azure OpenAI GPT-4, Anthropic Claude |
| **Orchestration** | LangChain, Semantic Kernel |
| **UI** | Copilot Studio, Custom React app |
| **Security** | Azure AD, API key management |

## 8. Key Takeaways

### What We Built

1. **Simple RAG** for policy question answering
2. **Function calling** pattern for data queries
3. **Intent classification** to route queries
4. **Conversational interface** for treasury team

### Production Enhancements

| Feature | Implementation |
|---------|---------------|
| **Real LLM** | Azure OpenAI / Anthropic API |
| **Vector Search** | Embeddings + similarity search |
| **Live Data** | SAP RFC, TMS API integration |
| **Security** | Azure AD auth, audit logging |
| **Feedback** | Thumbs up/down for improvement |

### Limitations

⚠️ AI assistants should **augment, not replace** human judgment
- Always verify critical information
- Use for routine queries, not final decisions
- Human review required for exceptions

---

*Author: Ozgur Guler (ozgur.guler1@gmail.com)*